In [1744]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns
import re
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
from math import sqrt

# Census Data

In [1745]:
x2011 = pd.read_csv('zip_codes_2011.csv').drop(['do_date'], axis = 1).dropna(how = 'all', axis = 1)
x2012 = pd.read_csv('zip_codes_2012.csv').drop(['do_date'], axis = 1).dropna(how = 'all', axis = 1)
x2013 = pd.read_csv('zip_codes_2013.csv').drop(['do_date'], axis = 1).dropna(how = 'all', axis = 1)
x2014 = pd.read_csv('zip_codes_2014.csv').drop(['do_date'], axis = 1).dropna(how = 'all', axis = 1)
x2015 = pd.read_csv('zip_codes_2015.csv').drop(['do_date'], axis = 1).dropna(how = 'all', axis = 1)
x2016 = pd.read_csv('zip_codes_2016.csv').drop(['do_date'], axis = 1).dropna(how = 'all', axis = 1)
x2017 = pd.read_csv('zip_codes_2017.csv').drop(['do_date'], axis = 1).dropna(how = 'all', axis = 1)

x2011.columns = x2011.columns.str.replace('geo_id', 'zip')
x2012.columns = x2012.columns.str.replace('geo_id', 'zip')
x2013.columns = x2013.columns.str.replace('geo_id', 'zip')
x2014.columns = x2014.columns.str.replace('geo_id', 'zip')
x2015.columns = x2015.columns.str.replace('geo_id', 'zip')
x2016.columns = x2016.columns.str.replace('geo_id', 'zip')
x2017.columns = x2017.columns.str.replace('geo_id', 'zip')

mutual_columns = set(x2011.columns).intersection(set(x2012.columns)).intersection(set(x2013.columns)).\
intersection(set(x2014.columns)).intersection(set(x2015.columns)).intersection(set(x2016.columns)).\
intersection(set(x2017.columns))


x2011 = x2011[mutual_columns].sort_values('zip').set_index('zip')
x2012 = x2012[mutual_columns].sort_values('zip').set_index('zip')
x2013 = x2013[mutual_columns].sort_values('zip').set_index('zip')
x2014 = x2014[mutual_columns].sort_values('zip').set_index('zip')
x2015 = x2015[mutual_columns].sort_values('zip').set_index('zip')
x2016 = x2016[mutual_columns].sort_values('zip').set_index('zip')
x2017 = x2017[mutual_columns].sort_values('zip').set_index('zip')

In [1746]:
x2011.columns = [i + '_2011' for i in x2011.columns]
x2012.columns = [i + '_2012' for i in x2012.columns] 
x2013.columns = [i + '_2013' for i in x2013.columns]
x2014.columns = [i + '_2014' for i in x2014.columns]
x2015.columns = [i + '_2015' for i in x2015.columns]
x2016.columns = [i + '_2016' for i in x2016.columns]
x2017.columns = [i + '_2017' for i in x2017.columns]

In [1747]:
census2013 = pd.concat([x2013,x2012,x2011], join = 'inner', axis = 1).reset_index()
census2014 = pd.concat([x2014,x2013,x2012], join = 'inner', axis = 1).reset_index()
census2015 = pd.concat([x2015,x2014,x2013], join = 'inner', axis = 1).reset_index()
census2016 = pd.concat([x2016,x2015,x2014], join = 'inner', axis = 1).reset_index()
census2017 = pd.concat([x2017,x2016,x2015], join = 'inner', axis = 1).reset_index()

In [1748]:
census2013['year'] = '2013'
census2014['year'] = '2014'
census2015['year'] = '2015'
census2016['year'] = '2016'
census2017['year'] = '2017'

In [1749]:
census2013.columns = census2013.columns.str.replace('_2013','_3y').str.replace('_2012','_4y').str.replace('_2011','_5y')
census2014.columns = census2014.columns.str.replace('_2014','_3y').str.replace('_2013','_4y').str.replace('_2012','_5y')
census2015.columns = census2015.columns.str.replace('_2015','_3y').str.replace('_2014','_4y').str.replace('_2013','_5y')
census2016.columns = census2016.columns.str.replace('_2016','_3y').str.replace('_2015','_4y').str.replace('_2014','_5y')
census2017.columns = census2017.columns.str.replace('_2017','_3y').str.replace('_2016','_4y').str.replace('_2015','_5y')

In [1750]:
census = pd.concat([census2013,census2014, census2015, census2016, census2017]).sort_values(['zip','year'])


In [1751]:
census

,zip,pop_in_labor_force_3y,population_3_years_over_3y,female_65_to_66_3y,rent_10_to_15_percent_3y,family_households_3y,female_60_to_61_3y,rent_20_to_25_percent_3y,median_income_3y,dwellings_5_to_9_units_3y,...,male_18_to_19_5y,occupation_sales_office_5y,two_or_more_races_pop_5y,owner_occupied_housing_units_lower_value_quartile_5y,income_per_capita_5y,female_22_to_24_5y,rent_over_50_percent_5y,poverty_5y,households_retirement_income_5y,year
0,601,5698,17854,173,98,4395,294,86,12041.0,153,...,351,846,0,59400.0,6967.0,278,244,10102,550,2013
0,601,6037,17543,165,88,4362,199,52,10833.0,162,...,347,894,0,70500.0,7283.0,261,303,10200,511,2014
0,601,6162,17469,181,134,4396,188,55,10816.0,169,...,318,878,0,62700.0,7380.0,260,334,10816,528,2015
0,601,6056,17398,183,121,4421,209,54,11507.0,162,...,295,806,0,65700.0,7229.0,302,345,10911,531,2016
0,601,5953,17133,165,117,4227,275,87,11757.0,193,...,274,863,0,61900.0,7453.0,286,425,10861,656,2017
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33119,99929,1181,2307,16,12,643,69,54,45649.0,36,...,15,340,233,95100.0,29918.0,30,37,203,199,2013
33119,99929,1115,2310,9,21,632,76,61,48153.0,54,...,21,231,228,94300.0,28323.0,48,39,178,279,2014
33119,99929,1175,2301,14,47,623,59,44,47941.0,72,...,32,277,192,88800.0,28399.0,36,50,215,293,2015
33119,99929,1181,2312,17,38,652,52,47,52452.0,80,...,19,245,185,103000.0,30638.0,42,27,228,345,2016


# Zillow Rent Index

In [1752]:
zillow = pd.read_csv('Zip_Zri_AllHomesPlusMultifamily.csv')
zillow.columns = zillow.columns.str.replace('RegionName', 'zip')

zri2011 = pd.concat([zillow.zip,zillow.iloc[:,11:23]], axis = 1).set_index('zip').dropna(how = 'all')
zri2012 = pd.concat([zillow.zip,zillow.iloc[:,23:35]], axis = 1).set_index('zip').dropna(how = 'all')
zri2013 = pd.concat([zillow.zip,zillow.iloc[:,35:47]], axis = 1).set_index('zip').dropna(how = 'all')
zri2014 = pd.concat([zillow.zip,zillow.iloc[:,47:59]], axis = 1).set_index('zip').dropna(how = 'all')
zri2015 = pd.concat([zillow.zip,zillow.iloc[:,59:71]], axis = 1).set_index('zip').dropna(how = 'all')
zri2016 = pd.concat([zillow.zip,zillow.iloc[:,71:83]], axis = 1).set_index('zip').dropna(how = 'all')
zri2017 = pd.concat([zillow.zip,zillow.iloc[:,83:95]], axis = 1).set_index('zip').dropna(how = 'all')
zri2018 = pd.concat([zillow.zip,zillow.iloc[:,95:107]], axis = 1).set_index('zip').dropna(how = 'all')
zri2019 = pd.concat([zillow.zip,zillow.iloc[:,107:119]], axis = 1).set_index('zip').dropna(how = 'all')

zri2011 = zri2011[zri2011.isna().sum(axis = 1) <= 3]
zri2012 = zri2012[zri2012.isna().sum(axis = 1) <= 3]
zri2013 = zri2013[zri2013.isna().sum(axis = 1) <= 3]
zri2014 = zri2014[zri2014.isna().sum(axis = 1) <= 3]
zri2015 = zri2015[zri2015.isna().sum(axis = 1) <= 3]
zri2016 = zri2016[zri2016.isna().sum(axis = 1) <= 3]
zri2017 = zri2017[zri2017.isna().sum(axis = 1) <= 3]
zri2018 = zri2018[zri2018.isna().sum(axis = 1) <= 3]
zri2019 = zri2019[zri2019.isna().sum(axis = 1) <= 3]

In [1753]:
zri2011 = zri2011.interpolate(method='linear',limit_direction='both', axis=1)
zri2012 = zri2012.interpolate(method='linear',limit_direction='both', axis=1)
zri2013 = zri2013.interpolate(method='linear',limit_direction='both', axis=1)
zri2014 = zri2014.interpolate(method='linear',limit_direction='both', axis=1)
zri2015 = zri2015.interpolate(method='linear',limit_direction='both', axis=1)
zri2016 = zri2016.interpolate(method='linear',limit_direction='both', axis=1)
zri2017 = zri2017.interpolate(method='linear',limit_direction='both', axis=1)
zri2018 = zri2018.interpolate(method='linear',limit_direction='both', axis=1)
zri2019 = zri2019.interpolate(method='linear',limit_direction='both', axis=1)

In [1754]:
zillow_monthly = pd.concat([zri2011, zri2012, zri2013, zri2014, zri2015, zri2016, zri2017, zri2018, zri2019], join = 'inner', axis = 1)



zillow_monthly.shape[0]



8752

In [1755]:
zri2011 = pd.DataFrame(zri2011.mean(axis = 1), columns = ['2011'])
zri2012 = pd.DataFrame(zri2012.mean(axis = 1), columns = ['2012'])
zri2013 = pd.DataFrame(zri2013.mean(axis = 1), columns = ['2013'])
zri2014 = pd.DataFrame(zri2014.mean(axis = 1), columns = ['2014'])
zri2015 = pd.DataFrame(zri2015.mean(axis = 1), columns = ['2015'])
zri2016 = pd.DataFrame(zri2016.mean(axis = 1), columns = ['2016'])
zri2017 = pd.DataFrame(zri2017.mean(axis = 1), columns = ['2017'])
zri2018 = pd.DataFrame(zri2018.mean(axis = 1), columns = ['2018'])
zri2019 = pd.DataFrame(zri2019.mean(axis = 1), columns = ['2019'])

### Response (Target Variable)

In [1756]:
zillow_mean = pd.concat([zri2013,zri2014,zri2015,zri2016,zri2017,zri2018,zri2019],join = 'inner', axis = 1).reset_index()
zillow_mean = pd.melt(zillow_mean, id_vars=['zip'], value_vars=['2013','2014','2015','2016','2017','2018','2019'],\
       var_name='year', value_name='ZRI+3').sort_values(['zip','year'])



In [1757]:
zillow_mean = zillow_mean[(zillow_mean['year'] == '2016') | (zillow_mean['year'] == '2017') | (zillow_mean['year'] == '2018') | (zillow_mean['year'] == '2019')]


In [1758]:
zillow_mean['year'] = zillow_mean.year.apply(lambda i:str(int(i)-3))

In [1759]:
zillow_mean

,zip,year,ZRI+3
34898,1001,2013,1380.166667
44761,1001,2014,1421.833333
54624,1001,2015,1338.500000
64487,1001,2016,1362.416667
38790,1005,2013,1268.833333
...,...,...,...
64639,99701,2016,1506.416667
34516,99705,2013,1502.666667
44379,99705,2014,1474.750000
54242,99705,2015,1479.583333


### Train Data (Zillow Rent Index from previous Years)

In [1760]:
zri_train_2013 = pd.concat([zri2013,zri2014,zri2015], join = 'inner', axis = 1)
zri_train_2014 = pd.concat([zri2014,zri2015,zri2016], join = 'inner', axis = 1)
zri_train_2015 = pd.concat([zri2015,zri2016,zri2017], join = 'inner', axis = 1)
zri_train_2016 = pd.concat([zri2016,zri2017,zri2018], join = 'inner', axis = 1)

In [1761]:
zri_train_2013.columns = zri_train_2013.columns.str.replace('2015','zri_1y').str.replace('2014','zri_2y').str.replace('2013','zri_3y')
zri_train_2014.columns = zri_train_2014.columns.str.replace('2016','zri_1y').str.replace('2015','zri_2y').str.replace('2014','zri_3y')
zri_train_2015.columns = zri_train_2015.columns.str.replace('2017','zri_1y').str.replace('2016','zri_2y').str.replace('2015','zri_3y')
zri_train_2016.columns = zri_train_2016.columns.str.replace('2018','zri_1y').str.replace('2017','zri_2y').str.replace('2016','zri_3y')

In [1762]:
zri_train_2013['year'] = '2013'
zri_train_2014['year'] = '2014'
zri_train_2015['year'] = '2015'
zri_train_2016['year'] = '2016'

In [1763]:
zri_train_2013 = zri_train_2013.reset_index()
zri_train_2014 = zri_train_2014.reset_index()
zri_train_2015 = zri_train_2015.reset_index()
zri_train_2016 = zri_train_2016.reset_index()

In [1764]:
zillow_train = pd.concat([zri_train_2013,zri_train_2014, zri_train_2015, zri_train_2016])


zillow_train

,zip,zri_3y,zri_2y,zri_1y,year
0,60657,1955.666667,2005.666667,2073.500000,2013
1,77494,1786.833333,1956.833333,1921.916667,2013
2,60614,2148.000000,2202.583333,2271.000000,2013
3,77449,1251.166667,1361.083333,1424.000000,2013
4,77084,1205.916667,1327.916667,1417.166667,2013
...,...,...,...,...,...
12043,26291,1021.583333,949.666667,918.416667,2016
12044,12720,1196.500000,1181.416667,1261.250000,2016
12045,26209,732.666667,762.666667,765.083333,2016
12046,41101,865.083333,851.666667,855.916667,2016


In [1765]:
total = zillow_train.merge(census, how = 'inner', on = ['zip','year']).merge(zillow_mean, how = 'inner', on = ['zip','year'])



In [1766]:
total = total.dropna(axis = 0).sort_values(['zip','year'])

In [1767]:
total

,zip,zri_3y,zri_2y,zri_1y,year,pop_in_labor_force_3y,population_3_years_over_3y,female_65_to_66_3y,rent_10_to_15_percent_3y,family_households_3y,...,male_18_to_19_5y,occupation_sales_office_5y,two_or_more_races_pop_5y,owner_occupied_housing_units_lower_value_quartile_5y,income_per_capita_5y,female_22_to_24_5y,rent_over_50_percent_5y,poverty_5y,households_retirement_income_5y,ZRI+3
5302,1001,1196.583333,1226.583333,1333.583333,2013,9607,16771,139,214,4085,...,195,2478,132,174200.0,28931.0,265,405,1428,1319,1380.166667
15153,1001,1226.583333,1333.583333,1380.166667,2014,9556,16635,161,198,4168,...,264,2499,146,169400.0,29158.0,307,382,1251,1347,1421.833333
25004,1001,1333.583333,1380.166667,1421.833333,2015,9600,16962,143,219,4080,...,161,2511,119,166400.0,28457.0,431,550,1537,1285,1338.500000
34855,1001,1380.166667,1421.833333,1338.500000,2016,9211,17029,260,169,3943,...,159,2367,133,167500.0,29444.0,415,600,1390,1240,1362.416667
38744,1005,1268.833333,1282.833333,1282.833333,2016,2988,5156,94,16,1150,...,104,625,1,161700.0,29004.0,43,65,224,280,1358.833333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35007,99701,1504.416667,1482.083333,1477.250000,2016,10452,17831,112,222,3807,...,214,2040,1680,132400.0,29907.0,416,956,2323,1589,1506.416667
4920,99705,1411.916667,1348.000000,1479.750000,2013,13267,22794,157,90,5780,...,363,2442,1225,151400.0,30177.0,449,244,1226,1640,1502.666667
14771,99705,1348.000000,1479.750000,1502.666667,2014,13457,22948,178,143,5747,...,382,2676,1473,145500.0,30358.0,495,356,1180,1554,1474.750000
24622,99705,1479.750000,1502.666667,1474.750000,2015,13745,23095,87,125,5740,...,356,2473,1585,151000.0,30768.0,437,352,1328,1788,1479.583333


# Machine Learning

## Baseline Model (just Zillow Rent Indexes from previous years)

In [1768]:
base_train = zri_train_2015.merge(zillow_mean, how = 'inner', on = ['zip','year'])
base_test = zri_train_2016.merge(zillow_mean, how = 'inner', on = ['zip','year'])

In [1769]:
base_x_train = base_train.iloc[:,1:4]
base_x_test = base_test.iloc[:,1:4]

In [1770]:
base_y_train = base_train.iloc[:,5]
base_y_test = base_test.iloc[:,5]

### Log Everything

In [1771]:
base_x_train = np.log(base_x_train+1)
base_x_test = np.log(base_x_test+1)
base_y_train = np.log(base_y_train)
base_y_test = np.log(base_y_test)

In [1772]:
base_model_ols = LinearRegression()
base_model_lasso = Lasso()
base_model_ridge = Ridge()
base_model_gradient = GradientBoostingRegressor()

In [1773]:
base_model_ols.fit(base_x_train, base_y_train)
base_model_lasso.fit(base_x_train, base_y_train)
base_model_ridge.fit(base_x_train, base_y_train)
base_model_gradient.fit(base_x_train, base_y_train)

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
                          learning_rate=0.1, loss='ls', max_depth=3,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=100,
                          n_iter_no_change=None, presort='auto',
                          random_state=None, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)

### Train Score

In [1774]:
print('Simple Linear Train Score: %.4f'%model_ols.score(x_train, y_train))
print('Lasso Train Score: %.4f'%model_lasso.score(x_train, y_train))
print('Ridge Train Score: %.4f'%model_ridge.score(x_train, y_train))
print('Gradient Regressor Train Score: %.4f'%model_gradient.score(x_train, y_train))

Simple Linear Train Score: 0.9939
Lasso Train Score: 0.0000
Ridge Train Score: 0.9939
Gradient Regressor Train Score: 0.9939


### Test Score

In [1775]:
print('Simple Linear Test Score: %.4f'%base_model_ols.score(base_x_test, base_y_test))
print('Lasso Test Score: %.4f'%base_model_lasso.score(base_x_test, base_y_test))
print('Ridge Test Score: %.4f'%base_model_ridge.score(base_x_test, base_y_test))
print('Gradient Regressor Test Score: %.4f'%base_model_gradient.score(base_x_test, base_y_test))

Simple Linear Test Score: 0.9921
Lasso Test Score: -0.0007
Ridge Test Score: 0.9919
Gradient Regressor Test Score: 0.9917


### RMSE Score

In [2051]:
print('Simple Linear RMSE Score: %.5f' % sqrt(mean_squared_error(np.exp(base_model_ols.predict(base_x_test)), np.exp(base_y_test))))
print('Lasso RMSE Score: %.5f' % sqrt(mean_squared_error(np.exp(base_model_lasso.predict(base_x_test)), np.exp(base_y_test))))
print('Ridge RMSE Score: %.5f' % sqrt(mean_squared_error(np.exp(base_model_ridge.predict(base_x_test)), np.exp(base_y_test))))
print('Gradient Regressor RMSE Score: %.5f' % sqrt(mean_squared_error(np.exp(base_model_gradient.predict(base_x_test)), np.exp(base_y_test))))


Simple Linear RMSE Score: 48.93983
Lasso RMSE Score: 610.14751
Ridge RMSE Score: 49.25927
Gradient Regressor RMSE Score: 51.05961


### Prediction Accuracy

In [1782]:
base_pred = pd.Series(np.exp(base_model_ols.predict(base_x_test)), index = base_y_test.index)
print("Average Prediction Accuracy: %.2f percent" %((1 - np.abs(np.exp(base_y_test)/base_pred-1)).mean() * 100))

Average Prediction Accuracy: 97.61 percent


## Census Model

In [1790]:
train = total[total['year'] != '2016']
train['zip'] = [str(i) for i in train.zip]
train['zip_year'] = train.zip + '_' + train.year
train = train.drop(['zip','year'], axis = 1).set_index('zip_year')

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [1791]:
test = total[total['year'] == '2016']
test['zip'] = [str(i) for i in test.zip]
test['zip_year'] = test.zip + '_' + test.year
test = test.drop(['zip','year'], axis = 1).set_index('zip_year')

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [1792]:
x_train = train.iloc[:,:-1]
y_train = train.iloc[:,-1]

In [1793]:
x_test = test.iloc[:,:-1]
y_test = test.iloc[:,-1]

### Log Everything

In [1794]:
x_train = np.log(x_train+1)
x_test = np.log(x_test+1)
y_train = np.log(y_train)
y_test = np.log(y_test)

In [1795]:
model_ols = LinearRegression()
model_lasso = Lasso()
model_ridge = Ridge()
model_gradient = GradientBoostingRegressor()

In [1796]:
model_ols.fit(x_train, y_train)
model_lasso.fit(x_train, y_train)
model_ridge.fit(x_train, y_train)
model_gradient.fit(x_train, y_train)

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
                          learning_rate=0.1, loss='ls', max_depth=3,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=100,
                          n_iter_no_change=None, presort='auto',
                          random_state=None, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)

### Train Score

In [1799]:
print('Simple Linear Train Score: %.4f'%model_ols.score(x_train, y_train))
print('Lasso Train Score: %.4f'%model_lasso.score(x_train, y_train))
print('Ridge Train Score: %.4f'%model_ridge.score(x_train, y_train))
print('Gradient Regressor Train Score: %.4f'%model_gradient.score(x_train, y_train))

Simple Linear Train Score: 0.9939
Lasso Train Score: 0.0000
Ridge Train Score: 0.9939
Gradient Regressor Train Score: 0.9939


### Test Score

In [1807]:
print('Simple Linear Test Score: %.4f'%model_ols.score(x_test, y_test))
print('Lasso Test Score: %.4f'%model_lasso.score(x_test, y_test))
print('Ridge Test Score: %.4f'%model_ridge.score(x_test, y_test))
print('Gradient Regressor Test Score: %.4f'%model_gradient.score(x_test, y_test))

Simple Linear Test Score: 0.9922
Lasso Test Score: -0.0030
Ridge Test Score: 0.9920
Gradient Regressor Test Score: 0.9916


### RMSE

In [1810]:
print('Simple Linear RMSE Score: %.5f' % sqrt(mean_squared_error(np.exp(model_ols.predict(x_test)), np.exp(y_test))))
print('Lasso RMSE Score: %.5f' % sqrt(mean_squared_error(np.exp(model_lasso.predict(x_test)), np.exp(y_test))))
print('Ridge RMSE Score: %.5f' % sqrt(mean_squared_error(np.exp(model_ridge.predict(x_test)), np.exp(y_test))))
print('Gradient Regressor RMSE Score: %.5f' % sqrt(mean_squared_error(np.exp(model_gradient.predict(x_test)), np.exp(y_test))))



Simple Linear Kaggle Score: 50.41683
Lasso Kaggle Score: 613.52357
Ridge Kaggle Score: 51.26327
Gradient Regressor Kaggle Score: 50.41806


### Prediction Accuracy

In [1159]:
pred = pd.Series(np.exp(model_ols.predict(x_test)), index = y_test.index)

In [1160]:
print("Average Prediction Accuracy: %.2f percent" %((1 - np.abs(np.exp(y_test)/pred-1)).mean() * 100))

Average Prediction Accuracy: 97.61 percent


## Feature Importance

In [1538]:
top_features = sorted(list(zip(x_train, model_gradient.feature_importances_)), key = lambda x: x[1], reverse = True)[0:10]

top_features

[('zri_1y', 0.995481160485091),
 ('owner_occupied_housing_units_median_value_5y', 0.001247875699917967),
 ('owner_occupied_housing_units_upper_value_quartile_5y',
  0.0011228900236242877),
 ('median_rent_5y', 0.00035347260452484237),
 ('median_rent_4y', 0.00026191308554064245),
 ('housing_built_2005_or_later_5y', 0.00023572833615591034),
 ('median_rent_3y', 0.00017116410379575337),
 ('housing_built_2005_or_later_4y', 0.0001508354983443681),
 ('income_per_capita_3y', 0.000112765257431231),
 ('percent_income_spent_on_rent_3y', 7.782620983487537e-05)]

## Model With Just Top Features Census

In [1811]:
top_x_train = x_train[[i[0] for i in top_features] + ['zri_2y'] + ['zri_3y']]
top_x_test = x_test[[i[0] for i in top_features] + ['zri_2y'] + ['zri_3y']]

In [1812]:
top_y_train = y_train.copy()
top_y_test = y_test.copy()

In [1813]:
top_model_ols = LinearRegression()
top_model_lasso = Lasso()
top_model_ridge = Ridge()
top_model_gradient = GradientBoostingRegressor()

top_model_ols.fit(top_x_train, top_y_train)
top_model_lasso.fit(top_x_train, top_y_train)
top_model_ridge.fit(top_x_train, top_y_train)
top_model_gradient.fit(top_x_train, top_y_train)

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
                          learning_rate=0.1, loss='ls', max_depth=3,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=100,
                          n_iter_no_change=None, presort='auto',
                          random_state=None, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)

### Train Score

In [1814]:
print('Simple Linear Train Score: %.4f'%top_model_ols.score(top_x_train, top_y_train))
print('Lasso Train Score: %.4f'%top_model_lasso.score(top_x_train, top_y_train))
print('Ridge Train Score: %.4f'%top_model_ridge.score(top_x_train, top_y_train))
print('Gradient Regressor Train Score: %.4f'%top_model_gradient.score(top_x_train, top_y_train))

Simple Linear Train Score: 0.9934
Lasso Train Score: 0.0000
Ridge Train Score: 0.9934
Gradient Regressor Train Score: 0.9936


### Test Score

In [1815]:
print('Simple Linear Test Score: %.4f'%top_model_ols.score(top_x_test, top_y_test))
print('Lasso Test Score: %.4f'%top_model_lasso.score(top_x_test, top_y_test))
print('Ridge Test Score: %.4f'%top_model_ridge.score(top_x_test, top_y_test))
print('Gradient Regressor Test Score: %.4f'%top_model_gradient.score(top_x_test, top_y_test))

Simple Linear Test Score: 0.9923
Lasso Test Score: -0.0030
Ridge Test Score: 0.9922
Gradient Regressor Test Score: 0.9915


### RMSE Score

In [1913]:
print('Simple Linear RMSE Score: %.5f' % sqrt(mean_squared_error(np.exp(top_model_ols.predict(top_x_test)), np.exp(top_y_test))))
print('Lasso RMSE Score: %.5f' % sqrt(mean_squared_error(np.exp(top_model_lasso.predict(top_x_test)), np.exp(top_y_test))))
print('Ridge RMSE Score: %.5f' % sqrt(mean_squared_error(np.exp(top_model_ridge.predict(top_x_test)), np.exp(top_y_test))))
print('Gradient Regressor RMSE Score: %.5f' % sqrt(mean_squared_error(np.exp(top_model_gradient.predict(top_x_test)), np.exp(top_y_test))))



Simple Linear RMSE Score: 47.63700
Lasso RMSE Score: 613.52357
Ridge RMSE Score: 47.84701
Gradient Regressor RMSE Score: 50.50655


### Prediction Accuracy

In [1820]:
top_pred = pd.Series(np.exp(top_model_ols.predict(top_x_test)), index = top_y_test.index)
print("Average Prediction Accuracy: %.2f percent" %((1 - np.abs(np.exp(top_y_test)/top_pred-1)).mean() * 100))

Average Prediction Accuracy: 97.64 percent


## Try deleting older data (2011 and 2012)

In [1169]:
new_train = total[total['year'] == '2015'].drop(['zip','year'], axis = 1)
new_test = total[total['year'] == '2016'].drop(['zip','year'], axis = 1)

In [1170]:
new_x_train = new_train.iloc[:,:-1]
new_x_test = new_test.iloc[:,:-1]

new_y_train = new_train.iloc[:,-1]
new_y_test = new_test.iloc[:,-1]

### Log Everything

In [1171]:
new_x_train = np.log(new_x_train+1)
new_x_test = np.log(new_x_test+1)
new_y_train = np.log(new_y_train)
new_y_test = np.log(new_y_test)

In [1172]:
new_model_ols = LinearRegression()
new_model_lasso = Lasso()
new_model_ridge = Ridge()
new_model_gradient = GradientBoostingRegressor()

new_model_ols.fit(new_x_train, new_y_train)
new_model_lasso.fit(new_x_train, new_y_train)
new_model_ridge.fit(new_x_train, new_y_train)
new_model_gradient.fit(new_x_train, new_y_train)

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
                          learning_rate=0.1, loss='ls', max_depth=3,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=100,
                          n_iter_no_change=None, presort='auto',
                          random_state=None, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)

### Train Score

In [1173]:
print('Simple Linear Train Score: %.4f'%new_model_ols.score(new_x_train, new_y_train))
print('Lasso Train Score: %.4f'%new_model_lasso.score(new_x_train, new_y_train))
print('Ridge Train Score: %.4f'%new_model_ridge.score(new_x_train, new_y_train))
print('Gradient Regressor Train Score: %.4f'%new_model_gradient.score(new_x_train, new_y_train))

Simple Linear Train Score: 0.9951
Lasso Train Score: 0.0000
Ridge Train Score: 0.9949
Gradient Regressor Train Score: 0.9953


### Test Score

In [1174]:
print('Simple Linear Test Score: %.4f'%new_model_ols.score(new_x_test, new_y_test))
print('Lasso Test Score: %.4f'%new_model_lasso.score(new_x_train, new_y_train))
print('Ridge Test Score: %.4f'%new_model_ridge.score(new_x_train, new_y_train))
print('Gradient Regressor Test Score: %.4f'%new_model_gradient.score(new_x_train, new_y_train))

Simple Linear Test Score: 0.9917
Lasso Test Score: 0.0000
Ridge Test Score: 0.9949
Gradient Regressor Test Score: 0.9953


### RMSE

In [2049]:
print('Simple Linear RMSE Score: %.5f' % sqrt(mean_squared_error(np.exp(new_model_ols.predict(new_x_test)), np.exp(new_y_test))))
print('Lasso RMSE Score: %.5f' % sqrt(mean_squared_error(np.exp(new_model_lasso.predict(new_x_test)), np.exp(new_y_test))))
print('Ridge RMSE Score: %.5f' % sqrt(mean_squared_error(np.exp(new_model_ridge.predict(new_x_test)), np.exp(new_y_test))))
print('Gradient Regressor RMSE Score: %.5f' % sqrt(mean_squared_error(np.exp(new_model_gradient.predict(new_x_test)), np.exp(new_y_test))))



Simple Linear RMSE Score: 50.52294
Lasso RMSE Score: 610.54952
Ridge RMSE Score: 50.60957
Gradient Regressor RMSE Score: 49.79466


### Prediction Accuracy

In [1176]:
new_pred = pd.Series(np.exp(new_model_gradient.predict(new_x_test)), index = new_y_test.index)
print("Average Prediction Accuracy: %.2f percent" %((1 - np.abs(np.exp(new_y_test)/new_pred-1)).mean() * 100))

Average Prediction Accuracy: 97.60 percent


# IRS Data

In [1821]:
irs2011 = pd.read_csv('11zpallagi.csv')
irs2012 = pd.read_csv('12zpallagi.csv')
irs2013 = pd.read_csv('13zpallagi.csv')
irs2014 = pd.read_csv('14zpallagi.csv')
irs2015 = pd.read_csv('15zpallagi.csv')
irs2016 = pd.read_csv('16zpallagi.csv')
irs2017 = pd.read_csv('17zpallagi.csv')

irs_mutual_columns = set(irs2013.columns).\
intersection(set(irs2014.columns)).intersection(set(irs2015.columns)).intersection(set(irs2016.columns)).\
intersection(set(irs2017.columns))

irs2013 = irs2013[irs_mutual_columns].drop(['STATEFIPS','STATE'], axis = 1)
irs2014 = irs2014[irs_mutual_columns].drop(['STATEFIPS','STATE'], axis = 1)
irs2015 = irs2015[irs_mutual_columns].drop(['STATEFIPS','STATE'], axis = 1)
irs2016 = irs2016[irs_mutual_columns].drop(['STATEFIPS','STATE'], axis = 1)
irs2017 = irs2017[irs_mutual_columns].drop(['STATEFIPS','STATE'], axis = 1)

irs2013.columns = irs2013.columns.str.replace('zipcode','zip')
irs2014.columns = irs2014.columns.str.replace('zipcode','zip')
irs2015.columns = irs2015.columns.str.replace('zipcode','zip')
irs2016.columns = irs2016.columns.str.replace('zipcode','zip')
irs2017.columns = irs2017.columns.str.replace('zipcode','zip')

irs2013 = irs2013.groupby('zip').sum()
irs2014 = irs2014.groupby('zip').sum()
irs2015 = irs2015.groupby('zip').sum()
irs2016 = irs2016.groupby('zip').sum()
irs2017 = irs2017.groupby('zip').sum()


In [1822]:
irs2013.columns = [i + '_2013' for i in irs2013.columns]
irs2014.columns = [i + '_2014' for i in irs2014.columns]
irs2015.columns = [i + '_2015' for i in irs2015.columns]
irs2016.columns = [i + '_2016' for i in irs2016.columns]
irs2017.columns = [i + '_2017' for i in irs2017.columns]

In [1823]:
taxes2015 = pd.concat([irs2015,irs2014,irs2013], join = 'inner', axis = 1).reset_index()
taxes2016 = pd.concat([irs2016,irs2015,irs2014], join = 'inner', axis = 1).reset_index()
taxes2017 = pd.concat([irs2017,irs2016,irs2015], join = 'inner', axis = 1).reset_index()

In [1824]:
taxes2015['year'] = '2015'
taxes2016['year'] = '2016'
taxes2017['year'] = '2017'

In [1825]:
taxes2015.columns = taxes2015.columns.str.replace('_2015','_3y').str.replace('_2014','_4y').str.replace('_2013','_5y')
taxes2016.columns = taxes2016.columns.str.replace('_2016','_3y').str.replace('_2015','_4y').str.replace('_2014','_5y')
taxes2017.columns = taxes2017.columns.str.replace('_2017','_3y').str.replace('_2016','_4y').str.replace('_2015','_5y')



In [1826]:
irs_train = zri_train_2015.merge(taxes2015, on  = ['zip','year']).merge(zillow_mean, how = 'inner', on = ['zip','year'])


In [1827]:
irs_test = zri_train_2016.merge(taxes2016, on  = ['zip','year']).merge(zillow_mean, how = 'inner', on = ['zip','year'])



In [1828]:
irs_train['zip'] = [str(i) for i in irs_train.zip]
irs_train['zip_year'] = irs_train.zip + '_' + irs_train.year
irs_train = irs_train.drop(['zip','year'], axis = 1).set_index('zip_year')

In [1829]:
irs_test['zip'] = [str(i) for i in irs_test.zip]
irs_test['zip_year'] = irs_test.zip + '_' + irs_test.year
irs_test = irs_test.drop(['zip','year'], axis = 1).set_index('zip_year')

In [1830]:
irs_train = irs_train[(irs_train < 0).sum(axis = 1) == 0]
irs_test = irs_test[(irs_test < 0).sum(axis = 1) == 0]

## Models Building (IRS)

In [1831]:
irs_x_train = irs_train.iloc[:,:-1]
irs_y_train = irs_train.iloc[:,-1]

In [1832]:
irs_x_test = irs_test.iloc[:,:-1]
irs_y_test = irs_test.iloc[:,-1]

### Log Evertyhing

In [1833]:
irs_x_train = np.log(irs_x_train+1)
irs_x_test = np.log(irs_x_test+1)

irs_y_train = np.log(irs_y_train)
irs_y_test = np.log(irs_y_test)

In [1834]:
irs_model_ols = LinearRegression()
irs_model_lasso = Lasso()
irs_model_ridge = Ridge()
irs_model_gradient = GradientBoostingRegressor()

irs_model_ols.fit(irs_x_train, irs_y_train)
irs_model_lasso.fit(irs_x_train, irs_y_train)
irs_model_ridge.fit(irs_x_train, irs_y_train)
irs_model_gradient.fit(irs_x_train, irs_y_train)

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
                          learning_rate=0.1, loss='ls', max_depth=3,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=100,
                          n_iter_no_change=None, presort='auto',
                          random_state=None, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)

### Train Score

In [1320]:
print('Simple Linear Train Score: %.4f'%irs_model_ols.score(irs_x_train, irs_y_train))
print('Lasso Train Score: %.4f'%irs_model_lasso.score(irs_x_train, irs_y_train))
print('Ridge Train Score: %.4f'%irs_model_ridge.score(irs_x_train, irs_y_train))
print('Gradient Regressor Train Score: %.4f'%irs_model_gradient.score(irs_x_train, irs_y_train))

Simple Linear Train Score: 0.9949
Lasso Train Score: 0.0000
Ridge Train Score: 0.9947
Gradient Regressor Train Score: 0.9949


### Test Score

In [1321]:
print('Simple Linear Test Score: %.4f'%irs_model_ols.score(irs_x_test, irs_y_test))
print('Lasso Test Score: %.4f'%irs_model_lasso.score(irs_x_test, irs_y_test))
print('Ridge Test Score: %.4f'%irs_model_ridge.score(irs_x_test, irs_y_test))
print('Gradient Regressor Test Score: %.4f'%irs_model_gradient.score(irs_x_test, irs_y_test))

Simple Linear Test Score: 0.9919
Lasso Test Score: -0.0009
Ridge Test Score: 0.9917
Gradient Regressor Test Score: 0.9917


### RMSE

In [1911]:
print('Simple Linear RMSE Score: %.5f' % sqrt(mean_squared_error(np.exp(irs_model_ols.predict(irs_x_test)), np.exp(irs_y_test))))
print('Lasso RMSE Score: %.5f' % sqrt(mean_squared_error(np.exp(irs_model_lasso.predict(irs_x_test)), np.exp(irs_y_test))))
print('Ridge RMSE Score: %.5f' % sqrt(mean_squared_error(np.exp(irs_model_ridge.predict(irs_x_test)), np.exp(irs_y_test))))
print('Gradient Regressor RMSE Score: %.5f' % sqrt(mean_squared_error(np.exp(irs_model_gradient.predict(irs_x_test)), np.exp(irs_y_test))))



Simple Linear RMSE Score: 51.39572
Lasso RMSE Score: 611.32553
Ridge RMSE Score: 52.43858
Gradient Regressor RMSE Score: 51.16028


### Accuracy

In [1842]:
irs_pred = pd.Series(np.exp(irs_model_ols.predict(irs_x_test)), index = irs_y_test.index)
print("Average Prediction Accuracy: %.2f percent" %((1 - np.abs(np.exp(irs_y_test)/irs_pred-1)).mean() * 100))

Average Prediction Accuracy: 97.57 percent


## Top Features From IRS

In [1955]:
sorted(list(zip(irs_x_train.columns, irs_model_gradient.feature_importances_)), key = lambda x: x[1], reverse = True)[0:10]



[('zri_1y', 0.997450481974797),
 ('zri_3y', 0.0009439677465665153),
 ('zri_2y', 0.0006102681556737682),
 ('A07260_3y', 8.886497565459614e-05),
 ('SCHF_4y', 7.253273690399848e-05),
 ('A02300_5y', 6.546155420672393e-05),
 ('A19300_5y', 5.2950426072523325e-05),
 ('A03210_4y', 4.4430101607438114e-05),
 ('SCHF_3y', 4.2061153776998855e-05),
 ('N18450_5y', 3.695146715501609e-05)]

In [1956]:
top_irs = sorted(list(zip(irs_x_train.columns, irs_model_gradient.feature_importances_)), key = lambda x: x[1], reverse = True)[0:20]



In [1957]:
top_irs_x_train = irs_x_train[[i[0] for i in top_irs]]
top_irs_x_test = irs_x_test[[i[0] for i in top_irs]]

In [1958]:
top_irs_y_train = irs_y_train.copy()
top_irs_y_test = irs_y_test.copy()

In [1959]:
top_irs_model_ols = LinearRegression()
top_irs_model_lasso = Lasso()
top_irs_model_ridge = Ridge()
top_irs_model_gradient = GradientBoostingRegressor()

top_irs_model_ols.fit(top_irs_x_train, top_irs_y_train)
top_irs_model_lasso.fit(top_irs_x_train, top_irs_y_train)
top_irs_model_ridge.fit(top_irs_x_train, top_irs_y_train)
top_irs_model_gradient.fit(top_irs_x_train, top_irs_y_train)

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
                          learning_rate=0.1, loss='ls', max_depth=3,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=100,
                          n_iter_no_change=None, presort='auto',
                          random_state=None, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)

### Train Score

In [1960]:
print('Simple Linear Train Score: %.4f'%top_irs_model_ols.score(top_irs_x_train, top_irs_y_train))
print('Lasso Train Score: %.4f'%top_irs_model_lasso.score(top_irs_x_train, top_irs_y_train))
print('Ridge Train Score: %.4f'%top_irs_model_ridge.score(top_irs_x_train, top_irs_y_train))
print('Gradient Regressor Train Score: %.4f'%top_irs_model_gradient.score(top_irs_x_train, top_irs_y_train))

Simple Linear Train Score: 0.9942
Lasso Train Score: 0.0000
Ridge Train Score: 0.9941
Gradient Regressor Train Score: 0.9948


### Test Score

In [1961]:
print('Simple Linear Test Score: %.4f'%top_irs_model_ols.score(top_irs_x_test, top_irs_y_test))
print('Lasso Test Score: %.4f'%top_irs_model_lasso.score(top_irs_x_test, top_irs_y_test))
print('Ridge Test Score: %.4f'%top_irs_model_ridge.score(top_irs_x_test, top_irs_y_test))
print('Gradient Regressor Test Score: %.4f'%top_irs_model_gradient.score(top_irs_x_test, top_irs_y_test))

Simple Linear Train Score: 0.9921
Lasso Train Score: -0.0009
Ridge Train Score: 0.9919
Gradient Regressor Train Score: 0.9918


### RMSE

In [1962]:
print('Simple Linear RMSE Score: %.5f' % sqrt(mean_squared_error(np.exp(top_irs_model_ols.predict(top_irs_x_test)), np.exp(top_irs_y_test))))
print('Lasso RMSE Score: %.5f' % sqrt(mean_squared_error(np.exp(top_irs_model_lasso.predict(top_irs_x_test)), np.exp(top_irs_y_test))))
print('Ridge RMSE Score: %.5f' % sqrt(mean_squared_error(np.exp(top_irs_model_ridge.predict(top_irs_x_test)), np.exp(top_irs_y_test))))
print('Gradient Regressor RMSE Score: %.5f' % sqrt(mean_squared_error(np.exp(top_irs_model_gradient.predict(top_irs_x_test)), np.exp(top_irs_y_test))))


Simple Linear Kaggle Score: 50.34790
Lasso Kaggle Score: 611.32553
Ridge Kaggle Score: 51.17912
Gradient Regressor Kaggle Score: 51.14325


### Prediction Accuracy

In [1963]:
top_irs_pred = pd.Series(np.exp(top_irs_model_ols.predict(top_irs_x_test)), index = top_irs_y_test.index)
print("Average Prediction Accuracy: %.2f percent" %((1 - np.abs(np.exp(top_irs_y_test)/top_irs_pred-1)).mean() * 100))

Average Prediction Accuracy: 97.60 percent


## Take Average of Census and IRS Predictions

### Full Census and Full IRS

In [1689]:
together = pd.concat([pred, irs_pred],join = 'inner', axis = 1)
together.columns = ['census','irs']

In [1855]:
together['average'] = (together.census + together.irs)/2

### RMSE

In [1864]:
together = pd.concat([together.average, np.exp(y_test)], join = 'inner', axis = 1)

In [1885]:
print("Average Root Mean Squared Error: %.3f" %sqrt(mean_squared_error(together.average, together['ZRI+3'])))

Average Root Mean Squared Error: 49.178


In [1886]:
print("Average Prediction Accuracy: %.2f percent" %((1 - np.abs(np.exp(y_test)/together.average - 1)).mean()*100))

Average Prediction Accuracy: 97.67 percent


### Top 10 Census and Top 10 IRS

In [1887]:
top_together = pd.concat([top_pred, top_irs_pred],join = 'inner', axis = 1)
top_together.columns = ['census','irs']

In [1888]:
top_together['average'] = (top_together.census + top_together.irs)/2

In [1982]:
top_together = pd.concat([top_together.average, np.exp(y_test)], join = 'inner', axis = 1)

In [1890]:
print("Average Root Mean Squared Error is: %.3f" %sqrt(mean_squared_error(top_together.average, top_together['ZRI+3'])))


Average Root Mean Squared Error is: 47.201


In [1891]:
print("Average Prediction Accuracy: %.2f percent" %((1 - np.abs(np.exp(y_test)/top_together.average - 1)).mean()*100))

Average Prediction Accuracy: 97.70 percent


## Full Model (top 10 irs + top 10 census)

In [2016]:
top_census_features = [i[0] for i in top_features]
top_census_features

['zri_1y',
 'owner_occupied_housing_units_median_value_5y',
 'owner_occupied_housing_units_upper_value_quartile_5y',
 'median_rent_5y',
 'median_rent_4y',
 'housing_built_2005_or_later_5y',
 'median_rent_3y',
 'housing_built_2005_or_later_4y',
 'income_per_capita_3y',
 'percent_income_spent_on_rent_3y']

In [2017]:
top_irs_features = [i[0] for i in top_irs][0:10]
top_irs_features

['zri_1y',
 'zri_3y',
 'zri_2y',
 'A07260_3y',
 'SCHF_4y',
 'A02300_5y',
 'A19300_5y',
 'A03210_4y',
 'SCHF_3y',
 'N18450_5y']

In [1965]:
full_top_features = list(set(top_census_features).union(set(top_irs_features)))

In [1966]:
total

,zip,zri_3y,zri_2y,zri_1y,year,pop_in_labor_force_3y,population_3_years_over_3y,female_65_to_66_3y,rent_10_to_15_percent_3y,family_households_3y,...,male_18_to_19_5y,occupation_sales_office_5y,two_or_more_races_pop_5y,owner_occupied_housing_units_lower_value_quartile_5y,income_per_capita_5y,female_22_to_24_5y,rent_over_50_percent_5y,poverty_5y,households_retirement_income_5y,ZRI+3
5302,1001,1196.583333,1226.583333,1333.583333,2013,9607,16771,139,214,4085,...,195,2478,132,174200.0,28931.0,265,405,1428,1319,1380.166667
15153,1001,1226.583333,1333.583333,1380.166667,2014,9556,16635,161,198,4168,...,264,2499,146,169400.0,29158.0,307,382,1251,1347,1421.833333
25004,1001,1333.583333,1380.166667,1421.833333,2015,9600,16962,143,219,4080,...,161,2511,119,166400.0,28457.0,431,550,1537,1285,1338.500000
34855,1001,1380.166667,1421.833333,1338.500000,2016,9211,17029,260,169,3943,...,159,2367,133,167500.0,29444.0,415,600,1390,1240,1362.416667
38744,1005,1268.833333,1282.833333,1282.833333,2016,2988,5156,94,16,1150,...,104,625,1,161700.0,29004.0,43,65,224,280,1358.833333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35007,99701,1504.416667,1482.083333,1477.250000,2016,10452,17831,112,222,3807,...,214,2040,1680,132400.0,29907.0,416,956,2323,1589,1506.416667
4920,99705,1411.916667,1348.000000,1479.750000,2013,13267,22794,157,90,5780,...,363,2442,1225,151400.0,30177.0,449,244,1226,1640,1502.666667
14771,99705,1348.000000,1479.750000,1502.666667,2014,13457,22948,178,143,5747,...,382,2676,1473,145500.0,30358.0,495,356,1180,1554,1474.750000
24622,99705,1479.750000,1502.666667,1474.750000,2015,13745,23095,87,125,5740,...,356,2473,1585,151000.0,30768.0,437,352,1328,1788,1479.583333


In [1967]:
### Join Data Sets

In [1968]:
census_x_train = total[total['year'] == '2015']
census_x_train['zip'] = [str(i) for i in census_x_train.zip]
census_x_train['zip_year'] = census_x_train.zip + '_' + census_x_train.year
census_x_train = census_x_train.drop(['zip','year'], axis = 1).set_index('zip_year')

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [1969]:
full_train = pd.concat([census_x_train.drop(['zri_1y','zri_2y','zri_3y','ZRI+3'], axis = 1),irs_train], join = 'inner', axis = 1)[full_top_features+['ZRI+3']]


In [1970]:
full_train.head()

,A03210_4y,income_per_capita_3y,median_rent_4y,N18450_5y,zri_2y,zri_3y,housing_built_2005_or_later_4y,A02300_5y,housing_built_2005_or_later_5y,SCHF_3y,SCHF_4y,zri_1y,A19300_5y,owner_occupied_housing_units_upper_value_quartile_5y,owner_occupied_housing_units_median_value_5y,median_rent_5y,percent_income_spent_on_rent_3y,median_rent_3y,A07260_3y,ZRI+3
zip_year,,,,,,,,,,,,,,,,,,,,
1001_2015,958.0,30044.0,800.0,190.0,1380.166667,1333.583333,59,5500.0,63,0.0,0.0,1421.833333,16302.0,265700.0,213000.0,802.0,31.5,809.0,126.0,1338.500000
1007_2015,940.0,34278.0,755.0,110.0,1527.250000,1599.375000,43,3936.0,15,50.0,50.0,1490.125000,20527.0,321500.0,260000.0,743.0,33.4,778.0,507.0,1504.500000
1013_2015,921.0,21477.0,730.0,100.0,1159.083333,1108.416667,84,7435.0,10,0.0,0.0,1226.416667,9581.0,208900.0,179800.0,689.0,30.9,731.0,43.0,1208.916667
1020_2015,1399.0,28032.0,710.0,190.0,1177.833333,1128.833333,16,9981.0,0,0.0,0.0,1243.750000,19001.0,230300.0,181400.0,699.0,28.9,718.0,62.0,1205.250000
1028_2015,971.0,38300.0,678.0,190.0,1554.500000,1567.833333,55,4260.0,39,0.0,0.0,1565.916667,23286.0,351100.0,256300.0,826.0,33.1,675.0,122.0,1483.666667


In [1971]:
census_x_test = total[total['year'] == '2016']
census_x_test['zip'] = [str(i) for i in census_x_test.zip]
census_x_test['zip_year'] = census_x_test.zip + '_' + census_x_test.year
census_x_test = census_x_test.drop(['zip','year'], axis = 1).set_index('zip_year')

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [1972]:
full_test = pd.concat([census_x_test.drop(['zri_1y','zri_2y','zri_3y','ZRI+3'], axis = 1), irs_test], join = 'inner', axis = 1)[full_top_features+['ZRI+3']]


In [1973]:
full_x_train = full_train.iloc[:,:-1]
full_x_test = full_test.iloc[:,:-1]

full_y_train = full_train.iloc[:,-1]
full_y_test = full_test.iloc[:,-1]

#### Log Everything

In [1974]:
full_x_train = np.log(full_x_train+1)
full_x_test = np.log(full_x_test+1)

full_y_train = np.log(full_y_train)
full_y_test = np.log(full_y_test)

In [1975]:
full_model_ols = LinearRegression()
full_model_ridge = Ridge()
full_model_gradient = GradientBoostingRegressor()

full_model_ols.fit(full_x_train, full_y_train)
full_model_ridge.fit(full_x_train, full_y_train)
full_model_gradient.fit(full_x_train, full_y_train)

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
                          learning_rate=0.1, loss='ls', max_depth=3,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=100,
                          n_iter_no_change=None, presort='auto',
                          random_state=None, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)

### R^2

In [1976]:
print('Simple Linear Test Score: %.4f'%full_model_ols.score(full_x_test, full_y_test))
print('Ridge Test Score: %.4f'%full_model_ridge.score(full_x_test, full_y_test))
print('Gradient Regressor Test Score: %.4f'%full_model_gradient.score(full_x_test, full_y_test))

Simple Linear Test Score: 0.9923
Ridge Test Score: 0.9921
Gradient Regressor Test Score: 0.9921


### RMSE

In [2044]:
print('Simple Linear RMSE Score: %.5f' % sqrt(mean_squared_error(np.exp(full_model_ols.predict(full_x_test)), np.exp(full_y_test))))
print('Ridge RMSE Score: %.5f' % sqrt(mean_squared_error(np.exp(full_model_ridge.predict(full_x_test)), np.exp(full_y_test))))
print('Gradient Regressor RMSE Score: %.5f' % sqrt(mean_squared_error(np.exp(full_model_gradient.predict(full_x_test)), np.exp(full_y_test))))


Simple Linear RMSE Score: 49.61277
Ridge RMSE Score: 50.37211
Gradient Regressor RMSE Score: 49.64466


### Top Features

In [2009]:
full_top = [i[0] for i in sorted(list(zip(full_x_train,full_model_gradient.feature_importances_)), key = lambda x: x[1], reverse = True)][:10]

In [2010]:
full_top

['zri_1y',
 'zri_2y',
 'median_rent_5y',
 'median_rent_4y',
 'zri_3y',
 'owner_occupied_housing_units_upper_value_quartile_5y',
 'income_per_capita_3y',
 'owner_occupied_housing_units_median_value_5y',
 'A03210_4y',
 'N18450_5y']

## Model with just top 10 most important features

In [2011]:
top_full_x_train = full_x_train[full_top]
top_full_x_test = full_x_test[full_top]

In [2012]:
top_full_y_train = full_y_train.copy()
top_full_y_test = full_y_test.copy()

In [2013]:
top_full_model_ols = LinearRegression()
top_full_model_ridge = Ridge()
top_full_model_gradient = GradientBoostingRegressor()

top_full_model_ols.fit(top_full_x_train, top_full_y_train)
top_full_model_ridge.fit(top_full_x_train, top_full_y_train)
top_full_model_gradient.fit(top_full_x_train, top_full_y_train)

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
                          learning_rate=0.1, loss='ls', max_depth=3,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=100,
                          n_iter_no_change=None, presort='auto',
                          random_state=None, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)

### R^2

In [2021]:
print('Simple Linear Test Score: %.4f'%top_full_model_ols.score(top_full_x_test, top_full_y_test))
print('Ridge Test Score: %.4f'%top_full_model_ridge.score(top_full_x_test, top_full_y_test))
print('Gradient Regressor Test Score: %.4f'%top_full_model_gradient.score(top_full_x_test, top_full_y_test))

Simple Linear Test Score: 0.9924
Ridge Test Score: 0.9923
Gradient Regressor Test Score: 0.9920


### RMSE

In [2041]:
print('Simple Linear RMSE Score: %.5f' % sqrt(mean_squared_error(np.exp(top_full_model_ols.predict(top_full_x_test)), np.exp(top_full_y_test))))
print('Ridge RMSE Score: %.5f' % sqrt(mean_squared_error(np.exp(top_full_model_ridge.predict(top_full_x_test)), np.exp(top_full_y_test))))
print('Gradient Regressor RMSE Score: %.5f' % sqrt(mean_squared_error(np.exp(top_full_model_gradient.predict(top_full_x_test)), np.exp(top_full_y_test))))


Simple Linear RMSE Score: 48.19237
Ridge RMSE Score: 48.78454
Gradient Regressor RMSE Score: 49.60003


## Voting Regressor (Census)

In [1374]:
from sklearn.ensemble import VotingRegressor

VR = VotingRegressor([('ols',model_ols),('rg', model_ridge),('gr',model_gradient)])

VR.fit(x_train, y_train)

VotingRegressor(estimators=[('ols',
                             LinearRegression(copy_X=True, fit_intercept=True,
                                              n_jobs=None, normalize=False)),
                            ('rg',
                             Ridge(alpha=1.0, copy_X=True, fit_intercept=True,
                                   max_iter=None, normalize=False,
                                   random_state=None, solver='auto',
                                   tol=0.001)),
                            ('gr',
                             GradientBoostingRegressor(alpha=0.9,
                                                       criterion='friedman_mse',
                                                       init=None,
                                                       learning_rate=0.1,
                                                       loss='ls', max_depth=3,
                                                       max_features=None,
                                  

### Train Score

In [1464]:
print('Voting Regressor Train Score: %.4f'%VR.score(x_train, y_train))

Voting Regressor Train Score: 0.9941


### Test Score

In [1465]:
print('Voting Regressor Test Score: %.4f'%VR.score(x_test, y_test))

Voting Regressor Test Score: 0.9924


### Kaggle Score

In [1466]:
print('Voting Regressor Kaggle Score: %.5f'  %sqrt(mean_squared_error(VR.predict(x_test), y_test)))

Voting Regressor Kaggle Score: 0.03147


### RMSE

In [1893]:
print('Voting Regressor Kaggle Score: %.5f'  %sqrt(mean_squared_error(np.exp(VR.predict(x_test)), np.exp(y_test))))

Voting Regressor Kaggle Score: 49.02019


### Accuracy

In [1452]:
VR_prd = pd.Series(np.exp(VR.predict(x_test)), index = y_test.index)

In [1471]:
print("Average Accuracy for VR Regressor is: %.2f" %((1 - np.abs(np.exp(y_test)/VR_prd - 1).mean())*100))

Average Accuracy for VR Regressor is: 97.65


## Voting Regressor (IRS)

In [1472]:
from sklearn.ensemble import VotingRegressor

irs_VR = VotingRegressor([('ols',irs_model_ols),('rg', irs_model_ridge),('gr',irs_model_gradient)])

irs_VR.fit(irs_x_train, irs_y_train)

VotingRegressor(estimators=[('ols',
                             LinearRegression(copy_X=True, fit_intercept=True,
                                              n_jobs=None, normalize=False)),
                            ('rg',
                             Ridge(alpha=1.0, copy_X=True, fit_intercept=True,
                                   max_iter=None, normalize=False,
                                   random_state=None, solver='auto',
                                   tol=0.001)),
                            ('gr',
                             GradientBoostingRegressor(alpha=0.9,
                                                       criterion='friedman_mse',
                                                       init=None,
                                                       learning_rate=0.1,
                                                       loss='ls', max_depth=3,
                                                       max_features=None,
                                  

In [1477]:
print('Voting Regressor Train Score: %.4f'%irs_VR.score(irs_x_train, irs_y_train))

Voting Regressor Train Score: 0.9951


In [1478]:
print('Voting Regressor Test Score: %.4f'%irs_VR.score(irs_x_test, irs_y_test))

Voting Regressor Test Score: 0.9921


In [1479]:
print('Voting Regressor Kaggle Score: %.5f'  %sqrt(mean_squared_error(irs_VR.predict(irs_x_test), irs_y_test)))

Voting Regressor Kaggle Score: 0.03206


In [2037]:
print('Average RMSE: %.5f'  %sqrt(mean_squared_error(np.exp(irs_VR.predict(irs_x_test)), np.exp(irs_y_test))))

Average RMSE: 50.68215


In [1558]:
irs_VR_prd = pd.Series(np.exp(irs_VR.predict(irs_x_test)), index = irs_y_test.index)


print("Average Accuracy for VR Regressor is: %.2f" %((1 - np.abs(np.exp(irs_y_test)/irs_VR_prd - 1).mean())*100))


Average Accuracy for VR Regressor is: 97.61


## Average of 2 Voting Regressors

In [1898]:
VR_together = pd.concat([VR_prd, irs_VR_prd], join = 'inner',axis = 1)

In [1899]:
VR_together.columns = ['census', 'irs']

In [1900]:
VR_together['average'] = (VR_together.census + VR_together.irs)/2

In [1904]:
VR_together = pd.concat([VR_together.average,np.exp(y_test)], join = 'inner', axis = 1)

In [1909]:
print("Average RMSE: %.3f" %sqrt(mean_squared_error(VR_together.average, VR_together['ZRI+3'])))

Average RMSE: 48.424


In [1910]:
(1 - np.abs(np.exp(y_test)/VR_together.average - 1)).mean()*100

97.69578632070164

## Voting Regressor for Combined Census and IRS

In [2038]:
from sklearn.ensemble import VotingRegressor

full_VR = VotingRegressor([('ols',full_model_ols),('rg', full_model_ridge),('gr', full_model_gradient)])

full_VR.fit(full_x_train, full_y_train)

VotingRegressor(estimators=[('ols',
                             LinearRegression(copy_X=True, fit_intercept=True,
                                              n_jobs=None, normalize=False)),
                            ('rg',
                             Ridge(alpha=1.0, copy_X=True, fit_intercept=True,
                                   max_iter=None, normalize=False,
                                   random_state=None, solver='auto',
                                   tol=0.001)),
                            ('gr',
                             GradientBoostingRegressor(alpha=0.9,
                                                       criterion='friedman_mse',
                                                       init=None,
                                                       learning_rate=0.1,
                                                       loss='ls', max_depth=3,
                                                       max_features=None,
                                  

### R^2

In [2039]:
print('Voting Regressor Test Score: %.4f'%full_VR.score(full_x_test, full_y_test))

Voting Regressor Test Score: 0.9923


### RMSE

In [2036]:
print('Average RMSE: %.5f'  %sqrt(mean_squared_error(np.exp(full_VR.predict(full_x_test)), np.exp(full_y_test))))


Average RMSE: 49.23186


## Voting Regressor for Top 10

In [2032]:
from sklearn.ensemble import VotingRegressor

top_full_VR = VotingRegressor([('ols',top_full_model_ols),('rg', top_full_model_ridge),('gr', top_full_model_gradient)])

top_full_VR.fit(top_full_x_train, top_full_y_train)


VotingRegressor(estimators=[('ols',
                             LinearRegression(copy_X=True, fit_intercept=True,
                                              n_jobs=None, normalize=False)),
                            ('rg',
                             Ridge(alpha=1.0, copy_X=True, fit_intercept=True,
                                   max_iter=None, normalize=False,
                                   random_state=None, solver='auto',
                                   tol=0.001)),
                            ('gr',
                             GradientBoostingRegressor(alpha=0.9,
                                                       criterion='friedman_mse',
                                                       init=None,
                                                       learning_rate=0.1,
                                                       loss='ls', max_depth=3,
                                                       max_features=None,
                                  

### R^2

In [2033]:
print('Voting Regressor Test Score: %.4f'%top_full_VR.score(top_full_x_test, top_full_y_test))

Voting Regressor Test Score: 0.9924


### RMSE

In [2043]:
print('Average RMSE: %.5f'  %sqrt(mean_squared_error(np.exp(top_full_VR.predict(top_full_x_test)), np.exp(top_full_y_test))))


Average RMSE: 48.34092


# Second Level Model (based on census and irs predictions)

In [1457]:
together = pd.concat([together,np.exp(y_test)], join = 'inner', axis = 1)

In [1458]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(together.iloc[:,:2], together.iloc[:,3], test_size=0.20, random_state=42)

In [1459]:
X_train = np.log(X_train)
X_test = np.log(X_test)
Y_train = np.log(Y_train)
Y_test = np.log(Y_test)

In [1460]:
second_model_ols = LinearRegression()
second_model_ridge = Ridge()
second_model_gradient = GradientBoostingRegressor()

second_model_ols.fit(X_train, Y_train)
second_model_ridge.fit(X_train, Y_train)
second_model_gradient.fit(X_train, Y_train)

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
                          learning_rate=0.1, loss='ls', max_depth=3,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=100,
                          n_iter_no_change=None, presort='auto',
                          random_state=None, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)

In [1461]:
print(second_model_ols.score(X_train, Y_train))
print(second_model_ridge.score(X_train, Y_train))
print(second_model_gradient.score(X_train, Y_train))

0.9928272033748813
0.9928261802322604
0.9935607575883438


In [1462]:
print(second_model_ols.score(X_test, Y_test))
print(second_model_ridge.score(X_test, Y_test))
print(second_model_gradient.score(X_test, Y_test))

0.9923927436413895
0.9923827091879923
0.9922117275494151


In [1463]:
print(sqrt(mean_squared_error(second_model_ols.predict(X_test), Y_test)))
print(sqrt(mean_squared_error(second_model_ridge.predict(X_test), Y_test)))
print(sqrt(mean_squared_error(second_model_gradient.predict(X_test), Y_test)))

0.03123234217610667
0.0312529341074772
0.03160174729959855


In [1484]:
(1 - np.abs(np.exp(Y_test)/pd.Series(np.exp(second_model_ols.predict(X_test)), index = Y_test.index) - 1).mean())*100



97.71715692029895

# Second Level Model (based on Voting Regressors of Census and IRS)

In [1594]:
VR_together = pd.concat([VR_together,np.exp(y_test)], join = 'inner', axis = 1)

In [1595]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(VR_together.iloc[:,:2], VR_together.iloc[:,3], test_size=0.20, random_state=42)

X_train = np.log(X_train)
X_test = np.log(X_test)
Y_train = np.log(Y_train)
Y_test = np.log(Y_test)

second_model_ols = LinearRegression()
second_model_ridge = Ridge()
second_model_gradient = GradientBoostingRegressor()

second_model_ols.fit(X_train, Y_train)
second_model_ridge.fit(X_train, Y_train)
second_model_gradient.fit(X_train, Y_train)

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
                          learning_rate=0.1, loss='ls', max_depth=3,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=100,
                          n_iter_no_change=None, presort='auto',
                          random_state=None, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)

In [1596]:
print(second_model_ols.score(X_train, Y_train))
print(second_model_ridge.score(X_train, Y_train))
print(second_model_gradient.score(X_train, Y_train))

0.9928287381021246
0.992826410275961
0.993618028628102


In [1597]:
print(second_model_ols.score(X_test, Y_test))
print(second_model_ridge.score(X_test, Y_test))
print(second_model_gradient.score(X_test, Y_test))

0.9924471754108709
0.9924333781323569
0.9920866483413247


In [1598]:
print(sqrt(mean_squared_error(second_model_ols.predict(X_test), Y_test)))
print(sqrt(mean_squared_error(second_model_ridge.predict(X_test), Y_test)))
print(sqrt(mean_squared_error(second_model_gradient.predict(X_test), Y_test)))

0.031120404078258723
0.031148816029516177
0.031854497675436674


In [1600]:
(1 - np.abs(np.exp(Y_test)/pd.Series(np.exp(second_model_ols.predict(X_test)), index = Y_test.index) - 1).mean())*100

97.73532606189075